In [ ]:
# pip install scikit-fuzzy

In [ ]:
import numpy as np
from skfuzzy import control as ctrl
from skfuzzy import membership as mf
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
# 1. Define Antecedent
pm25 = ctrl.Antecedent(np.arange(0, 501, 1), 'PM2.5')   # µg/m3
pm10 = ctrl.Antecedent(np.arange(0, 604, 1), 'PM10')    # µg/m3
no2  = ctrl.Antecedent(np.arange(0, 2050, 1), 'NO2')     # ppb
co   = ctrl.Antecedent(np.arange(0, 51, 0.1), 'CO')     # ppm
o3   = ctrl.Antecedent(np.arange(0, 0.2, 0.01), 'O3')      # ppm

# 2. Define Consequent
AQI = ctrl.Consequent(np.arange(0, 1000, 1), 'AQI')

In [ ]:
# 3. Membership function for each antecedent variable
# PM2.5
pm25['low']      = mf.trapmf(pm25.universe, [0,   7.08,  28.32, 35.4])
pm25['moderate'] = mf.trimf(pm25.universe, [35.5, 45.5, 55.4])
pm25['high']     = mf.trapmf(pm25.universe, [55.5, 144.48, 411.42, 500.4])

# PM10
pm10['low']      = mf.trapmf(pm10.universe, [0,   30.8,  123.3, 154])
pm10['moderate'] = mf.trimf(pm10.universe, [155, 204.5, 254])
pm10['high']     = mf.trapmf(pm10.universe, [255, 324.8, 534.2, 604])

# NO2
no2['low']      = mf.trapmf(no2.universe, [0,   20,  80, 100])
no2['moderate'] = mf.trimf(no2.universe, [101, 230.5, 360])
no2['high']     = mf.trapmf(no2.universe, [361, 698.6, 1711.4, 2049])

# CO
co['low']      = mf.trapmf(co.universe, [0,   1.88, 7.52, 9.4])
co['moderate'] = mf.trimf(co.universe, [9.5, 10.95,  12.4])
co['high']     = mf.trapmf(co.universe, [12.5, 20.08, 42.82, 50.2])

# O3
o3['low']      = mf.trapmf(o3.universe, [0,   0.014,  0.056, 0.07])
o3['moderate'] = mf.trimf(o3.universe, [0.071, 0.078, 0.085])
o3['high']     = mf.trapmf(o3.universe, [0.086, 0.109, 0.177, 0.2])

In [ ]:
# 4. AQI Membership Function
# Using EPA-like numeric ranges with small overlaps for smoothness
AQI['Good']     = mf.trapmf(AQI.universe, [0, 0, 30, 60])
AQI['Moderate'] = mf.trapmf(AQI.universe, [40, 55, 85, 110])
AQI['Unhealthy_SG'] = mf.trapmf(AQI.universe, [90, 105, 135, 160])
AQI['Unhealthy'] = mf.trapmf(AQI.universe, [140, 160, 185, 210])
AQI['Very_Unhealthy'] = mf.trapmf(AQI.universe, [195, 210, 255, 320])
AQI['Hazardous'] = mf.trapmf(AQI.universe, [300, 320, 500, 1000])

In [ ]:
# Visualize membership functions
AQI.view()
plt.show()

pm25.view()
pm10.view()
no2.view()
co.view()
o3.view()
plt.show()

In [ ]:
# 5. Rules (compact representative set, safety-first)
# ---- Helper Conditions for 'two or more' logic ----
two_or_more_moderate = (
    (pm10['moderate'] & no2['moderate']) |
    (pm10['moderate'] & co['moderate']) |
    (pm10['moderate'] & o3['moderate']) |
    (no2['moderate'] & co['moderate']) |
    (no2['moderate'] & o3['moderate']) |
    (co['moderate'] & o3['moderate'])
)

two_or_more_high = (
    (pm10['high'] & no2['high']) |
    (pm10['high'] & co['high']) |
    (pm10['high'] & o3['high']) |
    (no2['high'] & co['high']) |
    (no2['high'] & o3['high']) |
    (co['high'] & o3['high'])
)

In [ ]:
# 5. Rules (compact representative set, safety-first)
# -- Set 1: PM2.5 Heavy Bias --
rule1  = ctrl.Rule(pm25['low'] & pm10['low'] & no2['low'] & co['low'] & o3['low'], AQI['Good'])
rule2  = ctrl.Rule(pm25['low'] & two_or_more_moderate, AQI['Moderate'])
rule3  = ctrl.Rule(pm25['low'] & (pm10['moderate'] & no2['moderate'] & co['moderate'] & o3['moderate']), AQI['Unhealthy_SG'])
rule4  = ctrl.Rule(pm25['low'] & two_or_more_high, AQI['Unhealthy'])
rule5  = ctrl.Rule(pm25['low'] & (pm10['high'] & no2['high'] & co['high'] & o3['high']), AQI['Very_Unhealthy'])
rule6  = ctrl.Rule(pm25['moderate'] & pm10['low'] & no2['low'] & co['low'] & o3['low'], AQI['Moderate'])
rule7  = ctrl.Rule(pm25['moderate'] & two_or_more_moderate, AQI['Unhealthy_SG'])
rule8  = ctrl.Rule(pm25['moderate'] & (pm10['moderate'] & no2['moderate'] & co['moderate'] & o3['moderate']), AQI['Unhealthy'])
rule9  = ctrl.Rule(pm25['moderate'] & two_or_more_high, AQI['Very_Unhealthy'])
rule10 = ctrl.Rule(pm25['moderate'] & (pm10['high'] & no2['high'] & co['high'] & o3['high']), AQI['Hazardous'])
rule11 = ctrl.Rule(pm25['high'] & pm10['low'] & no2['low'] & co['low'] & o3['low'], AQI['Unhealthy_SG'])
rule12 = ctrl.Rule(pm25['high'] & two_or_more_moderate, AQI['Unhealthy'])
rule13 = ctrl.Rule(pm25['high'] & (pm10['moderate'] & no2['moderate'] & co['moderate'] & o3['moderate']), AQI['Very_Unhealthy'])
rule14 = ctrl.Rule(pm25['high'] & two_or_more_high, AQI['Hazardous'])

# -- Set 2: PM2.5 + O3 synergy --
rule15 = ctrl.Rule(pm25['moderate'] & o3['moderate'], AQI['Unhealthy_SG'])
rule16 = ctrl.Rule(pm25['high'] & o3['moderate'], AQI['Unhealthy'])
rule17 = ctrl.Rule(pm25['moderate'] & o3['high'], AQI['Unhealthy'])
rule18 = ctrl.Rule(pm25['high'] & o3['high'], AQI['Very_Unhealthy'])

# -- Set 3: PM10 + NO2 synergy --
rule19 = ctrl.Rule(pm10['moderate'] & no2['moderate'], AQI['Unhealthy_SG'])
rule20 = ctrl.Rule(pm10['high'] & no2['moderate'], AQI['Unhealthy'])
rule21 = ctrl.Rule(pm10['moderate'] & no2['high'], AQI['Unhealthy'])
rule22 = ctrl.Rule(pm10['high'] & no2['high'], AQI['Very_Unhealthy'])

# -- Set 4: CO + NO2 synergy --
rule23 = ctrl.Rule(co['moderate'] & no2['moderate'], AQI['Unhealthy_SG'])
rule24 = ctrl.Rule(co['high'] & no2['moderate'], AQI['Unhealthy'])
rule25 = ctrl.Rule(co['moderate'] & no2['high'], AQI['Unhealthy'])
rule26 = ctrl.Rule(co['high'] & no2['high'], AQI['Very_Unhealthy'])

# -- Set 5: PM2.5 + CO synergy --
rule27 = ctrl.Rule(pm25['moderate'] & co['moderate'], AQI['Unhealthy_SG'])
rule28 = ctrl.Rule(pm25['high'] & co['moderate'], AQI['Unhealthy'])
rule29 = ctrl.Rule(pm25['moderate'] & co['high'], AQI['Unhealthy'])
rule30 = ctrl.Rule(pm25['high'] & co['high'], AQI['Very_Unhealthy'])

# Combine rules
rules = [rule1, rule2, rule3, rule4, rule5, rule6, rule7, rule8, rule9, rule10,
         rule11, rule12, rule13, rule14, rule15, rule16, rule17, rule18,
         rule19, rule20, rule21, rule22, rule23, rule24, rule25, rule26, rule27, 
         rule28, rule29, rule30]

In [ ]:
# 6. Create control system & simulation using same pattern ---
aqi_ctrl = ctrl.ControlSystem(rules = rules)
aqi_train  = ctrl.ControlSystemSimulation(control_system = aqi_ctrl)

In [ ]:
# 7. Define sample input values and compute 
# Sample inputs (can change these to test different scenarios)
aqi_train.input['PM2.5'] = 40     # µg/m3
aqi_train.input['PM10']  = 169     # µg/m3
aqi_train.input['NO2']   = 90     # ppb
aqi_train.input['CO']    = 11    # ppm
aqi_train.input['O3']    = 0.06     # ppm

# compute
print(aqi_train.input)
aqi_train.compute()

# print the whole output dict and specific value
print("Full outputs:", aqi_train.output)
print("AQI numeric:", aqi_train.output['AQI'])

# display AQI membership activation for this sim
AQI.view(sim=aqi_train)
plt.show()

In [ ]:
# 8. 3D surface: AQI (z) over PM2.5 (x) and PM10 (y) with other inputs baseline ---
x, y = np.meshgrid(np.linspace(pm25.universe.min(), pm25.universe.max(), 60),
                   np.linspace(pm10.universe.min(), pm10.universe.max(), 60))
z_aqi = np.zeros_like(x, dtype=float)

# baseline values for other pollutants 
baseline = {'NO2': 20, 'CO': 1.0, 'O3': 30}

for i, r in enumerate(x):
    for j, c in enumerate(r):
        aqi_train.input['PM2.5'] = x[i, j]
        aqi_train.input['PM10']  = y[i, j]
        aqi_train.input['NO2']   = baseline['NO2']
        aqi_train.input['CO']    = baseline['CO']
        aqi_train.input['O3']    = baseline['O3']
        try:
            aqi_train.compute()
            z_aqi[i, j] = aqi_train.output['AQI']
        except:
            z_aqi[i, j] = float('inf')  # handle invalid computations

# Plot surface using your plot3d style
def plot3d(x, y, z, title="AQI Surface (PM2.5 vs PM10, others baseline)"):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # main 3D surface
    ax.plot_surface(x, y, z, rstride=1, cstride=1, cmap='viridis',
                    linewidth=0.4, antialiased=True)

    # contour projections on 3 planes for visual clarity
    ax.contourf(x, y, z, zdir='z', offset=np.nanmin(z)-10, cmap='viridis', alpha=0.5)
    ax.contourf(x, y, z, zdir='x', offset=x.max()*1.1, cmap='viridis', alpha=0.5)
    ax.contourf(x, y, z, zdir='y', offset=y.max()*1.1, cmap='viridis', alpha=0.5)

    # labels, viewing angle, and title
    ax.set_xlabel('PM2.5 (µg/m³)')
    ax.set_ylabel('PM10 (µg/m³)')
    ax.set_zlabel('AQI')
    ax.view_init(30, 200)
    plt.title(title)

    plt.show()

# display the plot
plot3d(x, y, z_aqi)